In [1]:
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'

In [3]:
import torch
torch.cuda.device_count()

2

In [5]:
from transformers import AutoTokenizer, RagRetriever, RagSequenceForGeneration
import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/rag-sequence-nq")
retriever = RagRetriever.from_pretrained(
    "facebook/rag-sequence-nq", index_name="custom", passages_path="/share/llm-verify/my_knowledge_dataset/my_knowledge_dataset", index_path="/share/llm-verify/my_knowledge_dataset/my_knowledge_dataset_hnsw_index.faiss"
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

In [16]:
# initialize with RagRetriever to do everything in one forward call
model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-nq", retriever=retriever)
model.cuda()

/home/vvjain3/.local/lib/python3.10/site-packages/transformers/models/bart/configuration_bart.py:179: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


RagSequenceForGeneration(
  (rag): RagModel(
    (question_encoder): DPRQuestionEncoder(
      (question_encoder): DPREncoder(
        (bert_model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0-11): 12 x BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_features=768, out_features=768, bias=True)
                    (value): Linear(in_features=768, out_features=768, bias=True)
                    (dropout): Dropout(p

In [18]:
inputs = tokenizer("What is Douglas Adams the author of?", return_tensors="pt")
# targets = tokenizer(text_target="In Paris, there are 10 million people.", return_tensors="pt")
input_ids = inputs["input_ids"].cuda()
# labels = targets["input_ids"].cuda()
# outputs = model(input_ids=input_ids, labels=labels)
outputs = model.generate(input_ids, num_beams=4, early_stopping=True)
print("Generated:", tokenizer.batch_decode(outputs, skip_special_tokens=True))

Generated: [" the hitchhiker's guide to the galaxy"]


In [22]:
# or use retriever separately
model1 = RagSequenceForGeneration.from_pretrained("facebook/rag-sequence-nq", use_dummy_dataset=True).cuda()

Some weights of the model checkpoint at facebook/rag-sequence-nq were not used when initializing RagSequenceForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.weight', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.bias']
- This IS expected if you are initializing RagSequenceForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagSequenceForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
# 1. Encode
question_hidden_states = model1.question_encoder(input_ids)[0]
# 2. Retrieve
docs_dict = retriever(input_ids.cpu().detach().numpy(), question_hidden_states.cpu().detach().numpy(), return_tensors="pt")
doc_scores = torch.bmm(
    question_hidden_states.unsqueeze(1), docs_dict["retrieved_doc_embeds"].float().transpose(1, 2).cuda()
).squeeze(1)
# 3. Forward to generator
outputs = model1.generate(
    context_input_ids=docs_dict["context_input_ids"].cuda(),
    context_attention_mask=docs_dict["context_attention_mask"].cuda(),
    doc_scores=doc_scores.cuda()
)
print("Generated:", tokenizer.batch_decode(outputs, skip_special_tokens=True))

Generated: [" the hitchhiker's guide to the galaxy"]


In [26]:
print("Context:", tokenizer.batch_decode(docs_dict["context_input_ids"], skip_special_tokens=True))

Context: [" Q42 / can be seen on display at The Centre for Computing History in Cambridge.In 2018, John Lloyd presented an hour-long episode of the BBC Radio Four documentary Archive on 4, discussing Adams' private papers, which are held at St John's College, Cambridge. The episode is available online.Travessa Douglas Adams, a street at 27°35′21.8″S 48°39′44.0″W in São José, Santa Catarina, Brazil is named in Adams's honour.In March 2021 Unbound announced a crowdfunder for 42: the wildly improbable ideas of Douglas Adams, on the 20th anniversary of his death, a book based on Adams's papers, edited by Kevin Jon Davies.The annual Douglas Adams // what is douglas adams the author of?", ' Q42 / Douglas Noël Adams (11 March 1952 – 11 May 2001) was an English author, humorist, and screenwriter, best known for The Hitchhiker\'s Guide to the Galaxy. Originally a 1978 BBC radio comedy, The Hitchhiker\'s Guide to the Galaxy developed into a "trilogy" of five books that sold more than 15\xa0milli